In [62]:
import json
import io
import re


In [5]:
with open ('../raw-files/all-party-information.json', 'r') as f:
    parties = json.load(f)['data_list']

In [9]:
'we have %d parties' % (len(parties))

'we have 104 parties'

In [88]:
party2candidates = {}
with open ('../raw-files/pm-candidates.json', 'r') as f:
    candidates = json.load(f)['Result']
print("We have %d PM candidates" % len(candidates))
for c in candidates:
    pno = c['PartyNumber']
    if pno not in party2candidates.keys():
        party2candidates[pno] = [c]
    else:
        party2candidates[pno].append(c)

We have 69 PM candidates


In [89]:
party2candidates[206]

[{'Sequence': 1,
  'Title': 'นาย',
  'FirstName': 'พงศา',
  'LastName': 'ชูแนม',
  'PartyNumber': 206,
  'PartyName': 'กรีน'}]

In [105]:
def write_line(s, template):
    s.write('{}\n\n'.format(template))
    
def parties_md_parties(f, n=5):

    for i, p in enumerate(parties[:n]):
        if n is not None and n < 10:
            print(p)
        write_func = lambda t: write_line(f, t.format(**p))
#         f.write(.format(**p))
        write_func('## %d {name_th} ({name_en})' % (i+1))
        write_func('#### หัวหน้าพรรค {leader}')
        write_func('**Donation code:** {donate_code}')
        write_func('**Address:** {address}')
        write_func('**Website:** {website}')
        write_func('**Policy**\n```\n{policy}\n```')

        if p['id_dga'] in party2candidates.keys():
            write_func('**PM Candidates**')
            for c in sorted(party2candidates[p['id_dga']], key=lambda x: x['Sequence']):
                write_func('{Sequence}. {Title} {FirstName} {LastName}'.format(**c))
        write_func('')
sio = io.StringIO()
parties_md_parties(sio, n=None)

In [106]:
with open('../raw-files/all-party-infomation-with-pm-candidates.md', 'w') as f:
    f.write(sio.getvalue())